## Importing & Reading Data

In [3]:
# Importing library that will be use
import numpy as np # For array manipulation
import pandas as pd # For dataframe manipulation
import requests # For requesting data from API
import json # For reading and formating json file
import re # For data clean

In [4]:
# Creating dataframe and take a look
card_list = pd.read_csv('yugioh.csv')
card_list

,card_name
0,Tornado Dragon
1,Exodia the Forbidden One
2,Dark Magician
3,Kuriboh
4,Obelisk The Tormentor
5,Pot of Greed
6,Yami
7,Mystical Elf
8,Monster Reborn
9,Slifer the Sky Dragon


## Data Preperation for Enriching the data

In [5]:
# Create a list that contain card name
card_list = card_list.card_name.to_list()
card_list

['Tornado Dragon',
 'Exodia the Forbidden One',
 'Dark Magician',
 'Kuriboh',
 'Obelisk The Tormentor',
 'Pot of Greed',
 'Yami',
 'Mystical Elf',
 'Monster Reborn',
 'Slifer the Sky Dragon']

Testing if data is available in Public API

In [6]:
API = 'https://db.ygoprodeck.com/api/v7/cardinfo.php'

params = {'name': ['Tornado Dragon']}
endpoint_get = API
r = requests.get(endpoint_get, params = params)

In [7]:
# See the result
r.json()

{'data': [{'id': 6983839,
   'name': 'Tornado Dragon',
   'type': 'XYZ Monster',
   'desc': '2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.',
   'atk': 2100,
   'def': 2000,
   'level': 4,
   'race': 'Wyrm',
   'attribute': 'WIND',
   'card_sets': [{'set_name': 'Battles of Legend: Relentless Revenge',
     'set_code': 'BLRR-EN084',
     'set_rarity': 'Secret Rare',
     'set_rarity_code': '(ScR)',
     'set_price': '4.65'},
    {'set_name': 'Duel Devastator',
     'set_code': 'DUDE-EN019',
     'set_rarity': 'Ultra Rare',
     'set_rarity_code': '(UR)',
     'set_price': '1.58'},
    {'set_name': 'Legendary Duelists: Synchro Storm',
     'set_code': 'LED8-EN055',
     'set_rarity': 'Common',
     'set_rarity_code': '(C)',
     'set_price': '0.96'},
    {'set_name': 'Maximum Crisis',
     'set_code': 'MACR-EN081',
     'set_rarity': 'Secret Rare',
     'set_rarity_code': '(ScR)',
     'set_pr

Let's try to loop fetch 10 cards

In [8]:
# Initiate dict to collect card info
str_from_dict=''

# Create the loop
for card in card_list:
    # Set the parameters of wanted card
    params = {'name': card}

    # Requesting the data
    endpoint_get = API
    r = requests.get(endpoint_get, params = params)
    d_card = r.json()

    # Creating a list of the data
    if str_from_dict == '':
        str_from_dict += '['
        str_from_dict+=json.dumps(d_card)  
    else:
        str_from_dict+=','+json.dumps(d_card)
        
# Var last stores every card JSON, in string format
last = str_from_dict + ']'

# Reload card info as JSON
data = json.loads(last)

In [9]:
#Check the data
data

[{'data': [{'id': 6983839,
    'name': 'Tornado Dragon',
    'type': 'XYZ Monster',
    'desc': '2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.',
    'atk': 2100,
    'def': 2000,
    'level': 4,
    'race': 'Wyrm',
    'attribute': 'WIND',
    'card_sets': [{'set_name': 'Battles of Legend: Relentless Revenge',
      'set_code': 'BLRR-EN084',
      'set_rarity': 'Secret Rare',
      'set_rarity_code': '(ScR)',
      'set_price': '4.65'},
     {'set_name': 'Duel Devastator',
      'set_code': 'DUDE-EN019',
      'set_rarity': 'Ultra Rare',
      'set_rarity_code': '(UR)',
      'set_price': '1.58'},
     {'set_name': 'Legendary Duelists: Synchro Storm',
      'set_code': 'LED8-EN055',
      'set_rarity': 'Common',
      'set_rarity_code': '(C)',
      'set_price': '0.96'},
     {'set_name': 'Maximum Crisis',
      'set_code': 'MACR-EN081',
      'set_rarity': 'Secret Rare',
      'set_rarity_

## Merge the data

Normalize the data and see the data

In [10]:
df = pd.json_normalize(data)
df

,data
0,"[{'id': 6983839, 'name': 'Tornado Dragon', 'ty..."
1,"[{'id': 33396948, 'name': 'Exodia the Forbidde..."
2,"[{'id': 46986421, 'name': 'Dark Magician', 'ty..."
3,"[{'id': 40640057, 'name': 'Kuriboh', 'type': '..."
4,"[{'id': 10000000, 'name': 'Obelisk the Torment..."
5,"[{'id': 55144522, 'name': 'Pot of Greed', 'typ..."
6,"[{'id': 59197169, 'name': 'Yami', 'type': 'Spe..."
7,"[{'id': 15025844, 'name': 'Mystical Elf', 'typ..."
8,"[{'id': 83764719, 'name': 'Monster Reborn', 't..."
9,"[{'id': 10000020, 'name': 'Slifer the Sky Drag..."


The data still not ready beacuse the JSON is nested in **data columns**

In [11]:
df = pd.json_normalize(data, 'data')
df.head(2)

,id,name,type,desc,atk,def,level,race,attribute,card_sets,card_images,card_prices,archetype,banlist_info.ban_tcg,banlist_info.ban_ocg,banlist_info.ban_goat
0,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,[{'set_name': 'Battles of Legend: Relentless R...,"[{'id': 6983839, 'image_url': 'https://images....","[{'cardmarket_price': '0.19', 'tcgplayer_price...",NaN,NaN,NaN,NaN
1,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ...",1000.0,1000.0,3.0,Spellcaster,DARK,"[{'set_name': 'Dark Beginning 1', 'set_code': ...","[{'id': 33396948, 'image_url': 'https://images...","[{'cardmarket_price': '0.46', 'tcgplayer_price...",Exodia,Limited,Limited,Limited


The data still nested in **card_sets, card_images, and card_prices columns**

In [12]:
# explode all columns with lists of dicts

df = df.explode("card_sets").reset_index().drop("index",1).explode("card_images").reset_index()\
    .drop("index",1).explode("card_prices").reset_index().drop("index",1)

# list of columns with dicts
cols_to_normalize = ['card_sets', 'card_images','card_prices']

# if there are keys, which will become column names, overlap with excising column names
# add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df[col])
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

# combine df with the normalized columns
df = pd.concat([df] + normalized, axis=1).drop(columns=cols_to_normalize)

C:\Users\madeba072283\AppData\Local\Temp\ipykernel_12360\2269917581.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.explode("card_sets").reset_index().drop("index",1).explode("card_images").reset_index()\
C:\Users\madeba072283\AppData\Local\Temp\ipykernel_12360\2269917581.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.explode("card_sets").reset_index().drop("index",1).explode("card_images").reset_index()\
C:\Users\madeba072283\AppData\Local\Temp\ipykernel_12360\2269917581.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.explode("card_sets").reset_index().drop("index",1).explode("card_images").reset_index()\


In [13]:
# The final dataframe
df.head(5)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
0,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.65,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
1,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(UR),1.58,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
2,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(C),0.96,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
3,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.49,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
4,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ...",1000.0,1000.0,3.0,Spellcaster,DARK,Exodia,...,(UR),40.1,33396948,https://images.ygoprodeck.com/images/cards/333...,https://images.ygoprodeck.com/images/cards_sma...,0.46,0.83,24.98,1.29,29.99


# Exploration

In [14]:
# See the shape of the data
print(f'There is {df.shape[0]} rows in the dataframe')
print(f'There is {df.shape[1]} columns in the dataframe')

There is 795 rows in the dataframe
There is 26 columns in the dataframe


In [15]:
# List of columns
print('List of all columns can be seen below:')
for colum in df.columns:
    print(colum)

List of all columns can be seen below:
id
name
type
desc
atk
def
level
race
attribute
archetype
banlist_info.ban_tcg
banlist_info.ban_ocg
banlist_info.ban_goat
card_sets_set_name
card_sets_set_code
card_sets_set_rarity
card_sets_set_rarity_code
card_sets_set_price
card_images_id
card_images_image_url
card_images_image_url_small
card_prices_cardmarket_price
card_prices_tcgplayer_price
card_prices_ebay_price
card_prices_amazon_price
card_prices_coolstuffinc_price


In [16]:
# See the null value
df.isnull().sum()

id                                  0
name                                0
type                                0
desc                                0
atk                                95
def                                95
level                              95
race                                0
attribute                          95
archetype                          97
banlist_info.ban_tcg              690
banlist_info.ban_ocg              690
banlist_info.ban_goat             690
card_sets_set_name                  0
card_sets_set_code                  0
card_sets_set_rarity                0
card_sets_set_rarity_code           0
card_sets_set_price                 0
card_images_id                      0
card_images_image_url               0
card_images_image_url_small         0
card_prices_cardmarket_price        0
card_prices_tcgplayer_price         0
card_prices_ebay_price              0
card_prices_amazon_price            0
card_prices_coolstuffinc_price      0
dtype: int64

In [17]:
# it can be seen that the atk, def, and, level of a spell card is null
df.loc[df['atk'].isnull(),:].head(5)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
642,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(MSR),4.98,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
643,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(R),5.47,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
644,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(R),3.94,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
645,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(SFR),3.54,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
646,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(SR),40.57,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99


In [18]:
# See list of spell card in the data
df.loc[df['atk'].isnull(),:]['name'].value_counts()

Monster Reborn    70
Pot of Greed      18
Yami               7
Name: name, dtype: int64

In [45]:
# To see the name of card_sets
df.loc[df['card_sets_set_name'].notna(),:]['card_sets_set_name'].value_counts()

Yugi's Legendary Decks                                             45
Legend of Blue Eyes White Dragon                                   44
Yu-Gi-Oh! The Dark Side of Dimensions Movie Pack Secret Edition    27
Starter Deck: Yugi                                                 26
Yu-Gi-Oh! Power of Chaos: Yugi the Destiny promotional cards       24
                                                                   ..
The Lost Art Promotion D                                            1
Duelist Pack: Kaiba                                                 1
Yu-Gi-Oh! GX Ultimate Beginner's Pack                               1
Duel Devastator                                                     1
2-Player Starter Deck: Yuya & Declan                                1
Name: card_sets_set_name, Length: 99, dtype: int64

In [19]:
# This means the card is not ban to be use in tcg competition
df.loc[df['banlist_info.ban_tcg'].isnull(),:].head(5)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
0,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.65,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
1,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(UR),1.58,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
2,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(C),0.96,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
3,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.49,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
21,46986421,Dark Magician,Normal Monster,''The ultimate wizard in terms of attack and d...,2500.0,2100.0,7.0,Spellcaster,DARK,Dark Magician,...,(UR),6.97,46986421,https://images.ygoprodeck.com/images/cards/469...,https://images.ygoprodeck.com/images/cards_sma...,0.02,0.54,10.95,14.45,4.99


In [37]:
#There is an enter in the description
print(df.iloc[1,3])

2 Level 4 monsters
Once per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.


In [34]:
# Make a function to remove enter in desc
def remove_enter(text):
    clean_text=text.strip().replace('\n', ' ')
    return clean_text


In [35]:
# Testing the function
test_text = df.iloc[1,3]
print(remove_enter(test_text))

2 Level 4 monsters Once per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.


In [38]:
# Apply into the dataframe
df['desc'] = df['desc'].apply(lambda x: remove_enter(x))

In [41]:
# Check the result
print(df.iloc[1,3])

2 Level 4 monsters Once per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.


In [42]:
df.head(2)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
0,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters Once per turn (Quick Effect...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.65,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
1,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters Once per turn (Quick Effect...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(UR),1.58,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.79
